## Geodepy Tutorial: Time-Dependent Datum Transformations

This tutorial demonstrates the use of GeodePy for transformations between time-dependent dynamic datums. We'll work through each of the steps required to tranform a coordinate in ITRF2005 at epoch 2000.0 to ITRF2014 at 2020.0 on the Australian Plate, then build a function containing these steps to simplify this process.

Start by importing the following Functions and Modules (datetime is used for the representation of epochs):

In [55]:
from datetime import date
from geodepy.constants import itrf14togda20, itrf14to05
from geodepy.transform import *

We'll define a point for transformation. For this exercise, we'll use a point called Bob which is in terms of ITRF2005 at epoch 2000.0 and is shown using a UTM projection. The parts below are (zone, easting, northing, ellipsoidal height):

In [56]:
bob = (53, 386353.2371, 7381852.2967, 587.5814)

#### Part 1: Getting Things in the Right Format
To convert between time-dependent datums, we need our coordinates to be in Cartesian (XYZ) format. To get Bob in this format, we start by converting the UTM part (zone, easting and northing) into Geographic (latitude and longitude) format:

In [57]:
lat, lon, psf, grid_conv = grid2geo(bob[0], bob[1], bob[2])
lat, lon

(-23.67011015602, 133.88552163574)

We can now use Bob's Latitude, Longitude and Ellipsoidal Height to calculate Cartesian Coordinates for Bob. These are still in terms of ITRF2005 at epoch 2000.0:

In [58]:
x, y, z = llh2xyz(lat, lon, bob[3])
x, y, z

(-4052042.792092285, 4212825.645301948, -2545098.3015855583)

#### Part 2: Moving to the Reference Epoch
Transformations between datums in GeodePy use Transformation Objects which are contained in the Constants module. These contain parameters used in Conformal (Helmert) 7 and 15 parameter transformations. To go between ITRF2005 and ITRF2014, we use `itrf14to05`, but because we're going the other way we use `-itrf14to05`:

In [59]:
-itrf14to05

Transformation: From 'ITRF2005' to 'ITRF2014'
Reference Epoch: datetime.date(2010, 1, 1)
  tx: -0.0026m + -0.0003m/yr
  ty: -0.001m + -0.0m/yr
  tz: 0.0023m + 0.0001m/yr
  sc: -0.00092ppm + -3e-05ppm/yr
  rx: -0.0" + -0.0"/yr
  ry: -0.0" + -0.0"/yr
  rz: -0.0" + -0.0"/yr

Before we can use this, we need to get our coordinates into the Reference Epoch of the transformation we plan to use (in this case, 2010.0 or the 1st Jan 2010). Because Bob is in Australia, we need to use the Australian Plate Motion Model to do this. It's parameters are in `itrf14togda20`:

In [60]:
itrf14togda20

Transformation: From 'ITRF2014' to 'GDA2020'
Reference Epoch: datetime.date(2020, 1, 1)
  tx: 0m + 0m/yr
  ty: 0m + 0m/yr
  tz: 0m + 0m/yr
  sc: 0ppm + 0ppm/yr
  rx: 0" + 0.00150379"/yr
  ry: 0" + 0.00118346"/yr
  rz: 0" + 0.00120716"/yr

It also has a reference epoch; for `itrf14togda20`, it's 2020.0. So to move coordinates from 2000.0 to 2010.0 (+10.0 years) using `itrf14togda20` we need to use `conform14` with a `to_epoch` 10.0 years past our reference epoch (2020.0 + 10.0 = 2030.0):

In [61]:
x, y, z

(-4052042.792092285, 4212825.645301948, -2545098.3015855583)

In [62]:
x2, y2, z2 = conform14(x, y, z, date(2030, 1, 1), itrf14togda20)
x2, y2, z2

(-4052042.3994570016, 4212825.69690139, -2545098.841287901)

We now have Bob's Cartesian Coordinates in ITRF2005 at epoch 2010.0!

#### Part 3: The Transformation
Now that we have Bob's coordinates in the reference epoch of `itrf14to05`, we can perform this transformation. Because we're not changing epochs in this part, we can use the `conform7` function which ignores the time-dependent parameters of our transformation:

In [63]:
x3, y3, z3 = conform7(x2, y2, z2, -itrf14to05)
x3, y3, z3

(-4052042.3983291225, 4212825.692025591, -2545098.8366464106)

This gives us Bob's Cartesian Coordinates in ITRF2014 at epoch 2010.0.
#### Part 4: Moving to the Final Epoch
The final tranformation is moving Bob's epoch to it's final destination (2020.0) using `itrf14togda20`. As the period of movement (2010.0 to 2020.0) is the same (+10.0 years), we use the same `to_epoch` of 2030.0 as in Part 2:

In [64]:
x4, y4, z4 = conform14(x3, y3, z3, date(2030, 1, 1), itrf14togda20)
x4, y4, z4

(-4052042.0056938054, 4212825.743624971, -2545099.376348734)

These coordinates are now in terms of ITRF2014 at epoch 2020.0
#### Part 5: Getting Things in their Original Format
To bring everything back into the format we started, we'll step through the process in Part 1 in reverse. So we'll start by converting our Cartesian Coordinates to Geographic format:

In [65]:
lat_end, lon_end, ell_ht_end = xyz2llh(x4, y4, z4)
lat_end, lon_end, ell_ht_end

(-23.670120761985057, 133.8855154120127, 587.5785029223189)

Next, we'll convert the latitude and longitude into UTM grid coordinates:

In [66]:
hem_end, zone_end, east_end, north_end, psf_end, grid_conv_end = geo2grid(lat_end, lon_end)
zone_end, east_end, north_end

(53, 386352.6116, 7381851.1174)

Combining this with our ellipsoidal height from above, we get the final coordinate:

In [67]:
zone_end, east_end, north_end, ell_ht_end

(53, 386352.6116, 7381851.1174, 587.5785029223189)

We can then compare the starting coordinate to the final coordinate and see the difference:

In [68]:
bob_end = (zone_end, east_end, north_end, ell_ht_end)
bob_end

(53, 386352.6116, 7381851.1174, 587.5785029223189)

In [69]:
bob

(53, 386353.2371, 7381852.2967, 587.5814)

#### Part 6: Simplifying steps into a single function
To wrap up this tutorial, we'll combine all of the steps above into a single function:

In [70]:
def bobtransform(zone, east, north, ell_ht):
    lat, lon, psf, grid_conv = grid2geo(zone, east, north)
    x, y, z = llh2xyz(lat, lon, ell_ht)
    x2, y2, z2 = conform14(x, y, z, date(2030, 1, 1), itrf14togda20)
    x3, y3, z3 = conform7(x2, y2, z2, -itrf14to05)
    x4, y4, z4 = conform14(x3, y3, z3, date(2030, 1, 1), itrf14togda20)
    lat_end, lon_end, ell_ht_end = xyz2llh(x4, y4, z4)
    hem_end, zone_end, east_end, north_end, psf_end, grid_conv_end = geo2grid(lat_end, lon_end)
    return zone_end, east_end, north_end, ell_ht_end

newbob = bobtransform(bob[0], bob[1], bob[2], bob[3])
newbob

(53, 386352.6116, 7381851.1174, 587.5785029223189)